In [24]:
import sys
sys.path.insert(0, "..")

In [25]:
from utils import access_raster
from glob import glob
import re
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from copy import deepcopy
from scipy.stats import gaussian_kde
import seaborn as sns
import random
from scipy import optimize

In [26]:
basepath = "/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/"
aspect_path = basepath + "Aspect"
hs_path = basepath + "HS"

is_tiff = lambda x: re.search(".tiff?$", x)

def get_dataframe(path, key): 
    files = filter(is_tiff, glob(path + "/**/*", recursive=True))
    df = pd.DataFrame(files, columns=[key])
    df['file'] = df[key].apply(lambda p: Path(p).parts[-2])
    df = df.set_index("file")
    return df

def get_array(raster): 
    array = raster['array'][0]
    nodata = raster['meta']['nodata']
    array[array==nodata]=1000
    array[array==-9999.] = 1000
    return array

In [27]:

df = get_dataframe(hs_path, "HS")
len(df)

16

In [28]:
def get_mapping(series):
    hs_raster = access_raster(series['HS'])
    hs_array = get_array(hs_raster)
    
    mapping = [ hs for hs_line in hs_array
                       for  hs in hs_line
                       if hs >= 0 and hs!=1000]
    
    return mapping

In [29]:

lengths = 0
lengthsx = 0
for index, series in df.iterrows(): 
    if series.name == "HS_Davos_20170316_2m_epsg2056":
        continue
    print("_"*100, "\n", series.name)
    mapping = get_mapping(series)
    random.shuffle(mapping)

    x = np.array(mapping[:10000])
    
    lengths+=len(x)
    lengthsx+=len(x[x<0.5])

____________________________________________________________________________________________________ 
 snowmap_20160309_ads_100_extent_maskiert_nosnow0_noausreisser
____________________________________________________________________________________________________ 
 HS_Saflischpass_20220512_1m_epsg2056
____________________________________________________________________________________________________ 
 HS_Laucherenalp_20220511_1m_epsg2056
____________________________________________________________________________________________________ 
 snowmap_2014_ads_100_extent_maskiert_nosnow0_noausreisser
____________________________________________________________________________________________________ 
 HS_Davos_20200406_2m_epsg2056
____________________________________________________________________________________________________ 
 HS_Davos_20211604_2m_epsg2056
____________________________________________________________________________________________________ 
 snowmap_2013_ads_80_exten

In [30]:
#WITHOUT 0.1
lengthsx, lengths, lengthsx/lengths*100

(22804, 150000, 15.202666666666667)

In [16]:
#WITHOUT 1
lengthsx, lengths, lengthsx/lengths*100

(7369, 150000, 4.9126666666666665)

In [13]:
path = "/home/abetatos/Escritorio/TFM_Dataset/Dataset_20230523/256pixel_4ro/Data"
import os
from tqdm import tqdm
import pickle

lengths = 0
lengthsx = 0

for filename in tqdm(os.listdir(path)):
    if "HS_Davos_20170316_2m_epsg2056" in filename: 
        continue

    filepath = os.path.join(path, filename)
    with open(filepath, 'rb') as f:
        array = pickle.load(f)

    x = array[-1][array[-1]>=0]
    lengths+=len(x)
    lengthsx+=len(x[x<0.2])

100%|██████████| 4050/4050 [00:00<00:00, 4374.79it/s]


In [14]:
#WITHOUT 1
lengthsx, lengths, lengthsx/lengths*100

(11897592, 178928626, 6.649350786385629)